In [ ]:
!pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp
import matplotlib.pyplot as plt

def solve_and_plot(rows: int, cols: int, solver_name: str = "CBC"):
    """
    Risolve il problema dell'ottimizzazione delle piantagioni di canna da zucchero.
    - Massimizza le celle UMIDE (HUMID).
    - A parità di celle umide, minimizza le sorgenti d'ACQUA (WATER).
    """

    # --- 1. Inizializzazione Solver ---
    solver = pywraplp.Solver.CreateSolver(solver_name)
    if not solver:
        raise RuntimeError(f"Solver {solver_name} non trovato.")

    # --- 2. Variabili Decisionali ---
    # w[r,c] = 1 se la cella è una sorgente d'ACQUA
    # h[r,c] = 1 se la cella è UMIDA (adiacente a un'acqua e non acqua essa stessa)
    w = {}
    h = {}
    for r in range(rows):
        for c in range(cols):
            w[r, c] = solver.IntVar(0, 1, f"w_{r}_{c}")
            h[r, c] = solver.IntVar(0, 1, f"h_{r}_{c}")

    # Helper per i vicini (ortogonali)
    def get_neighbors(r, c):
        nbs = []
        if r > 0: nbs.append((r - 1, c))
        if r < rows - 1: nbs.append((r + 1, c))
        if c > 0: nbs.append((r, c - 1))
        if c < cols - 1: nbs.append((r, c + 1))
        return nbs

    # --- 3. Vincoli ---
    for r in range(rows):
        for c in range(cols):
            # Vincolo 1: Una cella UMIDA non può essere contemporaneamente d'ACQUA
            solver.Add(h[r, c] <= 1 - w[r, c])

            # Vincolo 2: Una cella può essere UMIDA solo se ha almeno un vicino ACQUA
            nbs = get_neighbors(r, c)
            if nbs:
                # h[r,c] <= somma dei vicini w. Se la somma è 0, h deve essere 0.
                solver.Add(h[r, c] <= solver.Sum(w[nr, nc] for nr, nc in nbs))
            else:
                # Caso griglia 1x1
                solver.Add(h[r, c] == 0)

    # --- 4. Funzione Obiettivo ---
    weight_humid = (rows * cols) + 1

    sum_h = solver.Sum(h[r, c] for r in range(rows) for c in range(cols))
    sum_w = solver.Sum(w[r, c] for r in range(rows) for c in range(cols))

    # Obiettivo: Max (Peso * Umidità - Acqua)
    solver.Maximize(weight_humid * sum_h - sum_w)

    # --- 5. Risoluzione ---
    status = solver.Solve()

    if status not in (pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE):
        print("Nessuna soluzione trovata.")
        return

    # --- 6. Estrazione e Visualizzazione ---
    # 0: DRY (marrone), 1: WATER (blu), 2: HUMID (verde)
    res_grid = [[0] * cols for _ in range(rows)]
    count_h = 0
    count_w = 0

    for r in range(rows):
        for c in range(cols):
            if w[r, c].solution_value() > 0.5:
                res_grid[r][c] = 1
                count_w += 1
            elif h[r, c].solution_value() > 0.5:
                res_grid[r][c] = 2
                count_h += 1

    # Plot grafico
    plt.figure(figsize=(cols*0.7, rows*0.7))
    # Custom colormap: 0=Sienna, 1=DodgerBlue, 2=LimeGreen
    from matplotlib.colors import ListedColormap
    cmap = ListedColormap(['#8B4513', '#1E90FF', '#32CD32'])

    plt.imshow(res_grid, cmap=cmap, origin='upper')
    plt.title(f"Zucchero ILP: {count_h} Piante, {count_w} Acqua")
    plt.xticks(range(cols))
    plt.yticks(range(rows))
    plt.grid(visible=True, color='white', linestyle='--', linewidth=0.5, alpha=0.5)
    plt.show()

# Esempio di utilizzo:
solve_and_plot(6, 10)